# Setup

In [1]:
import torch
from torch import nn
from torch.utils.data import DataLoader
from torchvision import datasets
from torchvision.transforms import ToTensor

from torchmetrics import Accuracy
from torch.optim import Adam
from torch.optim.lr_scheduler import CosineAnnealingLR

from sklearn.metrics import classification_report
from sklearn.metrics import roc_curve, auc
from sklearn.metrics import roc_auc_score
from sklearn import metrics

In [2]:
import os
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import seaborn as sns

import random
from glob import glob
from tqdm import tqdm
from scipy.io import loadmat

import torch
from torch import nn
from torch.optim import Adam
from torch.optim.lr_scheduler import CosineAnnealingLR
from torch.utils.data import Dataset, DataLoader
import torch.nn.functional as F

from datetime import datetime

In [3]:
from IPython.display import clear_output
import os, sys, shutil
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np
from tqdm import tqdm, trange
from glob import glob
import random
import cv2 as cv
import torch.nn.functional as F
import json

In [4]:
main_data_dir =  "/media/mountHDD2"
print(os.getcwd())
save_dir = os.getcwd()
os.chdir("..")
os.chdir("..")
main_dir = os.getcwd() 
print(os.listdir(main_dir))
print(save_dir)

/media/mountHDD2/linh/git/HeartResearch/Chapman Dataset/2D_model/Proposed_model
['Data_Explore', 'Diagnostics.xlsx', '.ipynb_checkpoints', '2D_model', '1D_Model', '2D_Loss']
/media/mountHDD2/linh/git/HeartResearch/Chapman Dataset/2D_model/Proposed_model


In [5]:
data_dir = main_data_dir + "/ECGDataDenoised"
label_file = main_dir + "/Diagnostics.xlsx"

In [6]:
# data_df = pd.read_csv(data_dir_45 + "/MUSE_20180209_121900_35000.csv")
label_df = pd.read_excel(label_file)
label_df = label_df[['FileName', 'Rhythm']]

In [7]:
filename = label_df["FileName"].values.tolist()
print(len(filename))

10646


In [8]:
unique_values = np.unique(label_df["Rhythm"].values.tolist())
print(unique_values)

['AF' 'AFIB' 'AT' 'AVNRT' 'AVRT' 'SA' 'SAAWR' 'SB' 'SR' 'ST' 'SVT']


In [9]:
print(len(label_df["Rhythm"].values.tolist()))

10646


In [10]:
data_paths = [data_dir + f"/{x}.csv" for x in filename]
print(data_paths[0])
print(data_paths[0].split("/")[-1].split(".")[0])
print(len(data_paths))

/media/mountHDD2/ECGDataDenoised/MUSE_20180120_121711_19000.csv
MUSE_20180120_121711_19000
10646


In [11]:
# data_dir = "/media/mountHDD2/khoibaocon"
# print(os.listdir(data_dir))

In [12]:
# main_df = pd.read_csv(data_dir + "/Label.csv")
# main_df.shape

In [13]:
# single_main_df = main_df[main_df["Second_label"].isnull()]
# single_fns = single_main_df["Recording"].values.tolist()
# single_mat_paths = [data_dir + f"/alldata/{x}.mat" for x in single_fns]

In [14]:
ratio = [0.8, 0.1]

train_index = int(len(data_paths)*ratio[0])
valid_index = int(len(data_paths)*(ratio[0]+ratio[1]))
print(train_index)
train_mat_paths = data_paths[:train_index]
valid_mat_paths = data_paths[train_index:valid_index]
print(len(train_mat_paths))

train_label = label_df.iloc[:train_index,:]
valid_label = label_df.iloc[train_index:valid_index,:]

8516
8516


In [15]:
print(len(train_label))
print(len(valid_label))

8516
1065


In [37]:
label_df[label_df["FileName"] == filename]["Rhythm"].values

array(['SA', 'SA', 'SA', ..., 'AF', 'SA', 'ST'], dtype=object)

 # Data Loader

In [16]:
from torch.utils.data import Dataset, DataLoader
from torchvision import transforms, utils
from torch.nn.functional import one_hot

In [17]:
class HeartData(Dataset):
    def __init__(self, data_paths, label_df):
        self.data_paths = data_paths
        random.shuffle(self.data_paths)
        self.label_df = label_df
        normalize = transforms.Normalize(mean=[0.485, 0.456, 0.406],
                                     std=[0.229, 0.224, 0.225])
        self.transform = transforms.Compose([
            transforms.Resize((224, 224), antialias=None),
            normalize
        ])

    def __getitem__(self, idx):
        data_path = self.data_paths[idx]        
        # data = loadmat(data_path)['ECG'][0][0][2]
        data = pd.read_csv(data_path)
        data = data.values.T
        clip_data = data[:, 500:3000]
        clip_data = torch.tensor(clip_data, dtype=torch.float32)
        normalized_data = (clip_data - clip_data.min()) / (clip_data.max() - clip_data.min())
        grayscale_images = (normalized_data * 255)
        grayscale_images = grayscale_images.unsqueeze(0).unsqueeze(0) # (1, 1, h, w)
        resized_images = F.interpolate(grayscale_images, size=(9*4,2500), mode='bilinear', align_corners=True)
        resized_images = resized_images.squeeze(0).squeeze(0)
        torch_data = resized_images.unsqueeze(0).repeat(3, 1, 1)
        torch_data_resize = self.transform(torch_data)

        filename = data_path.split("/")[-1].split(".")[0]
        label = self.label_df[self.label_df["FileName"] == filename]["Rhythm"].values.item()

        # return torch_data_resize, label-1
        return torch_data_resize, label

    def __len__(self):
        return len(self.data_paths)    


In [18]:
train_ds = HeartData(train_mat_paths, label_df)
valid_ds = HeartData(valid_mat_paths, label_df)

In [19]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu", index = 0)
batch_size = 64

traindl = DataLoader(
    train_ds,
    batch_size=batch_size, 
    shuffle=True, 
    pin_memory=True, 
    num_workers=os.cpu_count()//2
)

validdl = DataLoader(
    valid_ds,
    batch_size=1, 
    shuffle=True, 
    pin_memory=True, 
    num_workers=os.cpu_count()//2
)

# Model 

In [20]:
from torchvision.models import efficientnet_b0, EfficientNet_B0_Weights
from torch import nn

class HeartModel(nn.Module):
    def __init__(self):
        super().__init__()
        self.ori_model = efficientnet_b0(weights = EfficientNet_B0_Weights.IMAGENET1K_V1)
        self.ori_model.classifier = nn.Sequential(
            nn.Dropout(0.2),
            nn.Linear(1280, 11),
            nn.Softmax(dim = 1)
        )
        
    def forward(self, x):        
        x = self.ori_model(x)
        return x

model = HeartModel()
# x = torch.randn((1, 3, 256, 512)).to("cuda")
# out = model(x)
# print(out.shape)

In [21]:
model

HeartModel(
  (ori_model): EfficientNet(
    (features): Sequential(
      (0): Conv2dNormActivation(
        (0): Conv2d(3, 32, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1), bias=False)
        (1): BatchNorm2d(32, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (2): SiLU(inplace=True)
      )
      (1): Sequential(
        (0): MBConv(
          (block): Sequential(
            (0): Conv2dNormActivation(
              (0): Conv2d(32, 32, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), groups=32, bias=False)
              (1): BatchNorm2d(32, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
              (2): SiLU(inplace=True)
            )
            (1): SqueezeExcitation(
              (avgpool): AdaptiveAvgPool2d(output_size=1)
              (fc1): Conv2d(32, 8, kernel_size=(1, 1), stride=(1, 1))
              (fc2): Conv2d(8, 32, kernel_size=(1, 1), stride=(1, 1))
              (activation): SiLU(inplace=True)
              (scale_

# Training

In [22]:
epoch = 150
lr = 0.001 # lr = 0.001 Acc: 0.821875 lr = 0.0005 Acc: 0.825 Focal_loss Acc: 0.828
best_acc = 0
best_ep = 0

model.to(device)
optimizer = Adam(model.parameters(), lr=lr)
scheduler = CosineAnnealingLR(optimizer=optimizer, T_max=epoch*len(traindl))
loss_fn = nn.CrossEntropyLoss()

In [23]:
# class FocalLoss(nn.Module):
#     def __init__(self, gamma=2, weight=None):
#         super(FocalLoss, self).__init__()
#         self.gamma = gamma
#         self.weight = weight
# #         self.reduction = reduction

#     def forward(self, inputs, target):
#         """
#         input: [N, C], float32
#         target: [N, ], int64
#         """
#         logpt = F.log_softmax(inputs, dim=1)
#         pt = torch.exp(logpt)
#         logpt = (1-pt)**self.gamma * logpt
#         loss = F.nll_loss(logpt, target, self.weight)
#         return loss
    
# focalloss_fn = FocalLoss()

# 0.1 - 0.001 - 82.1
# 0.1 - 0.0005 - 81.7

In [24]:
class FocalClassifierV0(nn.Module):
    def __init__(self, gamma=0.3):
        super().__init__()
        
        self.gamma = gamma
        self.act = nn.LogSoftmax(dim=1)
    
    def forward(self, pred, target):

        logits = self.act(pred)

        B, C = tuple(logits.size())

        entropy = torch.pow(1 - logits, self.gamma) * logits * F.one_hot(target, num_classes=C).float()

        return (-1 / B) * torch.sum(entropy)

focalloss_fn = FocalClassifierV0()
# checkpoint_folder = "run_efficientB0_heatmap_gamma0.5_lr0.0005"
checkpoint_folder = "run_proposed_gamma0.3_0.01"

In [25]:
def loss_plot(train_losses, val_losses, n_epochs, check_folder):
    now = datetime.now().strftime("%m-%d-%Y - %H-%M-%S")
    run_dir = save_dir + f"/{check_folder}"
    if not os.path.exists(run_dir):
        os.mkdir(run_dir)
    save_loss_dir = run_dir + "/save_losses"
    if not os.path.exists(save_loss_dir):
        os.mkdir(save_loss_dir)
    save_fig_losses = os.path.join(save_loss_dir, f"plot_losses_epoch{n_epochs}_{now}.png")  

    plt.figure()
    plt.plot(range(epoch), np.array(train_losses), label='Train Loss')
    plt.plot(range(epoch), np.array(val_losses), label='Test Loss')
    plt.xlabel('Epoch')
    # plt.xticks()
    plt.ylabel('Loss')
    plt.title('Training and Test Loss')
    plt.legend()
    plt.savefig(save_fig_losses)
    

def acc_plot(train_cls_acc, val_cls_acc, n_epochs, check_folder):
    now = datetime.now().strftime("%m-%d-%Y - %H-%M-%S")
    run_dir = os.getcwd() + f"/{check_folder}"
    if not os.path.exists(run_dir):
        os.mkdir(run_dir)
#         save_dir = run_dir + f"/{now}"
#         if not os.path.exists(save_dir):
#             os.mkdir(save_dir)
    save_acc_dir = run_dir + "/save_acc"
    if not os.path.exists(save_acc_dir):
        os.mkdir(save_acc_dir)
    save_fig_acc = os.path.join(save_acc_dir, 'plot_acc_epoch{}_{}.png'.format(n_epochs, now))  

    plt.figure()
    plt.plot(range(epoch), np.array(train_cls_acc), label='Train Accuracy')
    plt.plot(range(epoch), np.array(val_cls_acc), label='Test Accuracy')
    plt.xlabel('Epoch')
    # plt.xticks
    plt.ylabel('Accuracy')
    plt.title('Training and Test Accuracy')
    plt.legend()
    plt.savefig(save_fig_acc)

def checkpoint(valid_class_acc, 
               val_total_loss,
               old_valid_class_acc,
               old_valid_loss,
               epoch, 
               model,
               optimizer,
               check_folder
#                    logs
              ):

    if valid_class_acc >= old_valid_class_acc and val_total_loss <= old_valid_loss:
        old_valid_class_acc = valid_class_acc
        old_valid_loss = val_total_loss
        save_dict = {
            'epoch': epoch,
            'model_state_dict': model.state_dict(),
            'optimizer_state_dict': optimizer.state_dict(),
            'loss': val_total_loss,
            'test_acc': valid_class_acc
        }

     # Saving best model
        now = datetime.now().strftime("%m-%d-%Y - %H-%M-%S")
        run_dir = os.getcwd() + f"/{check_folder}"
        if not os.path.exists(run_dir):
            os.mkdir(run_dir)
    #         save_dir = run_dir + f"/{now}"
    #         if not os.path.exists(save_dir):
    #             os.mkdir(save_dir)
        save_best_model_dir = run_dir + "/save_best_model"
        if not os.path.exists(save_best_model_dir):
            os.mkdir(save_best_model_dir)
        save_best_model_path = save_best_model_dir + f"/{save_dict['loss']:>7f}_{save_dict['test_acc']:>7f}_{now}.pt"
        torch.save(save_dict, save_best_model_path)
        
def classification_report_csv(report, auc, check_folder):
    now = datetime.now().strftime("%m-%d-%Y - %H-%M-%S")
    run_dir = save_dir + f"/{check_folder}"
    if not os.path.exists(run_dir):
        os.mkdir(run_dir)
    save_report_dir = run_dir + "/save_classification_report"
    if not os.path.exists(save_report_dir):
        os.mkdir(save_report_dir)
        
    report_data = report['macro avg']
    del report_data['support']
    report_data.update({'auc': auc})
#     print(type(report_data))
#     print(report_data)

#     dataframe = pd.DataFrame.from_dict(report_data, orient='index')
#     save_report_file = save_report_dir + f"/classification_report_{now}.csv"
#     dataframe.to_csv(save_report_file, index = False)
    with open(save_report_dir + f"/cls_report_{now}.json", "w") as outfile: 
        json.dump(report_data, outfile)
    
def acc_loss_json(log_dict, check_folder):
    now = datetime.now().strftime("%m-%d-%Y - %H-%M-%S")
    run_dir = save_dir + f"/{check_folder}"
    if not os.path.exists(run_dir):
        os.mkdir(run_dir)
    save_json_dir = run_dir + "/save_acc_loss_json"
    if not os.path.exists(save_json_dir):
        os.mkdir(save_json_dir) 
    save_acc_loss_file = save_json_dir + f"/acc_loss_{now}.json"
    with open(save_acc_loss_file, "w") as outfile: 
        json.dump(log_dict, outfile)


In [26]:
log_dict = {
    "train": {
        "acc": [],
        "loss": []
    },
    "valid": {
        "acc": [],
        "loss": []
    }
}

In [27]:
# Training

class_la = np.unique(label_df["Rhythm"].values.tolist())
train_losses = []
train_acc_plot = []
val_losses = []
val_acc_plot = []
old_valid_class_acc = 0
old_valid_loss = 1e23
for e in range(epoch):
    model.train()
    print(f"Epoch: {e+1}")
    batch_cnt = 0
    total_loss = 0
    correct = 0
    for batch, (train_sig, train_label) in tqdm(enumerate(traindl)):
        batch_cnt = batch
        train_sig = train_sig.to(device)
        train_label = train_label.to(device)
        
        pred = model(train_sig)
        loss = focalloss_fn(pred, train_label)
        
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        
        scheduler.step()
        
        total_loss += loss.item()
        correct += (pred.argmax(1) == train_label).type(torch.float).sum().item()
        
    total_loss /= batch_cnt
    correct /= len(traindl.dataset)
    log_dict["train"]["loss"].append(total_loss)
    log_dict["train"]["acc"].append(correct)
    
    print(f"train loss: {total_loss} - train acc: {100*correct}")

# Valid
    batch_cnt = 0
    val_total_loss = 0
    val_correct = 0
    model.eval()
    y_true_list = [] 
    pred_list = []
    with torch.no_grad():
        for batch, (valid_sig, valid_label) in tqdm(enumerate(validdl)):
            batch_cnt = batch
            valid_label = valid_label.to(device)
            
            valid_sig = valid_sig.to(device)  
            pred = model(valid_sig)
            pred_pos = pred.argmax(1)
            
            y_true_list.append(valid_label)
            pred_list.append(pred_pos)
            
            loss = loss_fn(pred, valid_label)
            val_total_loss += loss.item()
            val_correct += (pred.argmax(1) == valid_label).type(torch.float).sum().item()
            
        val_total_loss /= batch_cnt
        val_correct /= len(validdl.dataset)
        log_dict["valid"]["loss"].append(val_total_loss)
        log_dict["valid"]["acc"].append(val_correct)

        if val_correct > best_acc:
            best_acc = val_correct
            best_ep = e 

        print(f"valid loss: {val_total_loss} - valid acc: {100*val_correct}")
#         checkpoint(valid_class_acc = val_correct, 
#                    val_total_loss = val_total_loss,
#                    old_valid_class_acc = old_valid_class_acc ,
#                    old_valid_loss = old_valid_loss,
#                    epoch = e, 
#                    model = model,
#                    optimizer = optimizer,
#                    check_folder = checkpoint_folder)
        if val_correct >= old_valid_class_acc and val_total_loss <= old_valid_loss:
            old_valid_class_acc = val_correct
            old_valid_loss = val_total_loss
            save_dict = {
                'epoch': e,
                'model_state_dict': model.state_dict(),
                'optimizer_state_dict': optimizer.state_dict(),
                'loss': val_total_loss,
                'test_acc': val_correct
            }

         # Saving best model
            now = datetime.now().strftime("%m-%d-%Y - %H-%M-%S")
            run_dir = os.getcwd() + f"/{checkpoint_folder}"
            if not os.path.exists(run_dir):
                os.mkdir(run_dir)
            save_best_model_dir = run_dir + "/save_best_model"
            if not os.path.exists(save_best_model_dir):
                os.mkdir(save_best_model_dir)
            save_best_model_path = save_best_model_dir + f"/{save_dict['loss']:>7f}_{save_dict['test_acc']:>7f}_{now}.pt"
            torch.save(save_dict, save_best_model_path)
        
print(f"Best acuracy: {best_acc} at epoch {best_ep}")

y_true = torch.cat(y_true_list).cpu().numpy()
pred = torch.cat(pred_list).cpu().numpy()
# print(type(y_true))
print(y_true)
print(f"pred: {pred}")

# Save loss and acc to json file
acc_loss_json(log_dict, check_folder = checkpoint_folder)
    
# Save plot acc and loss
# loss_plot(train_losses = train_losses, 
#           val_losses = val_losses, n_epochs = e , 
#           check_folder = checkpoint_folder)
# acc_plot(train_cls_acc = train_acc_plot , val_cls_acc = val_acc_plot, 
#                   n_epochs = e, check_folder = checkpoint_folder)

# AUC
fpr, tpr, thresholds = metrics.roc_curve(y_true, pred, pos_label = 0)
# print(fpr)
# print(f"tpr: {tpr}")
auc1 = metrics.auc(fpr, tpr)

# print(f"hihi {auc1}")
# print(type(auc1))

# Classification report
for i in range (len(class_la)):
    class_la[i] = str(class_la[i])

reports = classification_report(y_true, pred, target_names=class_la, output_dict=True) 
# print(reports)
classification_report_csv(report = reports, auc = auc1, check_folder = checkpoint_folder)

# json_object = json.dumps(reports, indent = 4) 
# print(json_object)
# ROC Curve
# fpr = [0] * 9
# tpr = [0] * 9
# thresholds = [0] * 9
# auc_score = [0] * 9
 
# for i in range(9):
#     fpr[i], tpr[i], thresholds[i] = roc_curve(y_true[:, i],
#                                               pred[:, i])
#     auc_score[i] = auc(fpr[i], tpr[i])
# print(auc_score)
# for i in range(9):
#     r2 = roc_auc_score(y_true, pred, average='micro', multi_class='ovr')
#     print("The ROC AUC score of "+ class_la[i] +" is: "+str(r2))

                    # Compute ROC curve and ROC area for each class
# class_fpr = {}
# class_tpr = {}
# class_roc_auc = dict()
# for i in range(9):
#     class_fpr[i], class_tpr[i], _ = roc_curve(valid_label[:, i].cpu(), pred[:, i].cpu(), drop_intermediate=False)
#     roc_auc[i] = auc(class_fpr[i], class_tpr[i])

# now = datetime.now().strftime("%m-%d-%Y - %H-%M-%S")
# run_dir = save_dir + f"/{check_folder}"
# if not os.path.exists(run_dir):
#     os.mkdir(run_dir)
# save_ROC_dir = run_dir + f"/save_ROC_{now}"
# if not os.path.exists(save_ROC_dir):
#     os.mkdir(save_ROC_dir)

# plt.plot(class_fpr[0], class_tpr[0],'turquoise',label='1: ROC curve of class 1 (area = %0.2f)' % roc_auc[0])
# plt.plot(class_fpr[1], class_tpr[1],'peachpuff',label='2: ROC curve of class 2 (area = %0.2f)' % roc_auc[1])
# plt.plot(class_fpr[2], class_tpr[2],'paleturquoise',label='3: ROC curve of class 3 (area = %0.2f)' % roc_auc[2])
# plt.plot(class_fpr[3], class_tpr[3],'pink',label='4: ROC curve of class 4 (area = %0.2f)' % roc_auc[3])
# plt.plot(class_fpr[4], class_tpr[4],'lightcoral',label='5: ROC curve of class 5 (area = %0.2f)' % roc_auc[4])
# plt.plot(class_fpr[5], class_tpr[5],'peachpuff',label='6: ROC curve of class 6 (area = %0.2f)' % roc_auc[5])
# plt.plot(class_fpr[6], class_tpr[6],'steelblue',label='7: ROC curve of class 7 (area = %0.2f)' % roc_auc[6])
# plt.plot(class_fpr[7], class_tpr[7],'forestgreen',label='8: ROC curve of class 8 (area = %0.2f)' % roc_auc[6])
# plt.plot(class_fpr[8], class_tpr[8],'darkslategray',label='9: ROC curve of class 9 (area = %0.2f)' % roc_auc[6])

# plt.plot([0, 1], [0, 1], 'k--')
# plt.xlim([-0.1, 1.1])
# plt.ylim([-0.1, 1.1])
# plt.xlabel('False Positive Rate')
# plt.ylabel('True Positive Rate')
# plt.title('Receiver operating characteristic of lead')
# plt.legend(loc="lower right")
# plt.savefig(save_ROC_dir)



Epoch: 1


0it [00:11, ?it/s]


AttributeError: 'list' object has no attribute 'to'

In [ ]:
# print(reports.keys())
# print(reports['macro avg'])

# Evaluation